In [ ]:
import numpy as np
import pandas as pd
import pickle, zlib
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
import scipy.cluster.hierarchy as sch
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim import corpora, models
from collections import Counter
# Show graph
%matplotlib inline
import matplotlib.pyplot as plt
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
import gensim.corpora as corpora
import random
from sklearn.manifold import TSNE

In [ ]:
def get_values(d):
    #map districtid label to names
    df = pd.read_excel('Files/ADI.xlsx')
    census = list(df['census_code'])
    dist_names = list(df['name'])
    keyss = list(d.keys())
    values = [i for i in keyss]
    for i in range(len(keyss)):
        if keyss[i] in census:
            values[i] = dist_names[census.index(keyss[i])]
    return values

In [ ]:
# Find median vector for each district
from collections import defaultdict
import statistics 
def find_median(vecs):
    for i in range(50):
        med.append(statistics.median(arr[:,i]))
    return med

def ret_val():
    return []

def get_vectors(temp_vector,temp_distt):
    d = defaultdict(ret_val)
    for i in range(len(temp_vector)):
        d[temp_distt[i]].append(temp_vector[i])

    vectors = []
    for i in d.keys():
        vectors.append(np.array(find_median(d[i])))
    return d,vectors

In [ ]:
def ret_val_2():
    return None

def get_vectors_dist(temp_vector,temp_distt):
    d = defaultdict(ret_val_2)
    for i in range(len(temp_vector)):
        d[temp_distt[i]]=(temp_vector[i])

    vectors = []
    for i in d.keys():
        vectors.append(np.array(d[i]))
    return d,vectors

In [ ]:
# K-Means Clustering - Elbow curve.
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist
#X = np.array(vectors)
# T-sne based on articles
import seaborn as sns

def kmeans(vectors):
    X_embedded = TSNE(n_components=2).fit_transform(np.array(vectors))
    #pca = PCA(n_components=2)
    #principalComponents = pca.fit_transform(np.array(vectors))
    #X = principalComponents
    X = X_embedded
    '''distortions = [] 
    inertias = [] 
    mapping1 = {} 
    mapping2 = {} 
    K = range(1,10) 

    for k in K: 
        #Building and fitting the model 
        kmeanModel = KMeans(n_clusters=k).fit(X)  

        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'),axis=1)) / X.shape[0]) 
        inertias.append(kmeanModel.inertia_) 

        mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                     'euclidean'),axis=1)) / X.shape[0] 
        mapping2[k] = kmeanModel.inertia_ 

    plt.plot(K, distortions, 'bx-') 
    plt.xlabel('Values of K') 
    plt.ylabel('Distortion') 
    plt.title('The Elbow Method using Distortion') 
    plt.show()
    '''
    labels = KMeans(n_clusters=3).fit(X).labels_
    
    return X,labels

In [ ]:
import mplcursors
def plot_cluster(X,labels,values,titletext,path,l=False):
    fig, ax = plt.subplots()
    if l:
        ax.scatter(X[:, 0], X[:, 1], c=labels,s=100, cmap='viridis')
    else:
        ax.scatter(X[:, 0], X[:, 1],s=100)
    plt.title(titletext)
    xx = list(X[:, 0])
    yy = list(X[:, 1])
    mplcursors.cursor(multiple = True).connect(
        "add", lambda sel: sel.annotation.set_text(
              values[sel.target.index]
    ))
    
    old_x = old_y = 1e9 # make an impossibly large initial offset
    thresh = .1 #make a distance threshold

    '''for label, x, y in zip(values, X[:, 0], X[:, 1]):
        #calculate distance
        d = ((x-old_x)**2+(y-old_y)**2)**(.5)

        #if distance less than thresh then flip the arrow
        flip = 1
        if d < .1: flip=-2

        plt.annotate(
            label,
            xy = (x, y), xytext = (-20*flip, 20*flip),
            textcoords = 'offset points', ha = 'right', va = 'bottom',
            bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
            arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
        old_x = x
        old_y = y
    '''
    
    if l:
        for i, txt in enumerate(values):
            ax.annotate(txt,(xx[i], yy[i]))
    plt.show()
    #plt.savefig(path+titletext+'.png')
    

In [ ]:
from collections import Counter
datasets = ['dataset_agriculture', 'dataset_development', 'dataset_environment', 'dataset_industrialization', 'dataset_lifestyle']
models = ['model_agriculture', 'model_development', 'model_environment', 'model_industrialization', 'model_lifestyle']

for dataset, model in zip(datasets,models):

    # Printing the collection name.
    collection_name = dataset[8:]
    print('\nCollection:',collection_name.capitalize())

    # Loading the dataset and the model from the drive.
    file = open('Datasets/'+dataset, 'rb')
    dataset = pickle.loads(zlib.decompress(pickle.load(file)))
    file.close()
    model = Doc2Vec.load('Models/'+model)

    # Collecting the article_ids, and corresponding article_vectors for each class.
    temp_ids = [[] for _ in range(9)]
    temp_dist = [[] for _ in range(9)]
    temp_titles = [[] for _ in range(9)]
    temp_vectors = [[] for _ in range(9)]
    temp_datasets = [[] for _ in range(9)]
    for i in dataset:
        if i[6]=='Unemp' and i[-1]=='Average':
            if i[0] not in temp_ids[0]:
                temp_ids[0].append(i[0])
                temp_dist[0].append(i[-5])
                temp_titles[0].append(i[1])
                temp_vectors[0].append(model.docvecs[i[0]])
        if i[6]=='Unemp' and i[-1]=='Slow':
            if i[0] not in temp_ids[1]:
                temp_ids[1].append(i[0])
                temp_dist[1].append(i[-5])
                temp_titles[1].append(i[1])
                temp_vectors[1].append(model.docvecs[i[0]])
        if i[6]=='Unemp' and i[-1]=='Fast':
            if i[0] not in temp_ids[2]:
                temp_ids[2].append(i[0])
                temp_dist[2].append(i[-5])
                temp_titles[2].append(i[1])
                temp_vectors[2].append(model.docvecs[i[0]])
        if i[6]=='Agri' and i[-1]=='Average':
            if i[0] not in temp_ids[3]:
                temp_ids[3].append(i[0])
                temp_dist[3].append(i[-5])
                temp_titles[3].append(i[1])
                temp_vectors[3].append(model.docvecs[i[0]])
        if i[6]=='Agri' and i[-1]=='Slow':
            if i[0] not in temp_ids[4]:
                temp_ids[4].append(i[0])
                temp_dist[4].append(i[-5])
                temp_titles[4].append(i[1])
                temp_vectors[4].append(model.docvecs[i[0]])
        if i[6]=='Agri' and i[-1]=='Fast':
            if i[0] not in temp_ids[5]:
                temp_ids[5].append(i[0])
                temp_dist[5].append(i[-5])
                temp_titles[5].append(i[1])
                temp_vectors[5].append(model.docvecs[i[0]])
        if i[6]=='Non Agri' and i[-1]=='Average':
            if i[0] not in temp_ids[6]:
                temp_ids[6].append(i[0])
                temp_dist[6].append(i[-5])
                temp_titles[6].append(i[1])
                temp_vectors[6].append(model.docvecs[i[0]])
        if i[6]=='Non Agri' and i[-1]=='Slow':
            if i[0] not in temp_ids[7]:
                temp_ids[7].append(i[0])
                temp_dist[7].append(i[-5])
                temp_titles[7].append(i[1])
                temp_vectors[7].append(model.docvecs[i[0]])
        if i[6]=='Non Agri' and i[-1]=='Fast':
            if i[0] not in temp_ids[8]:
                temp_ids[8].append(i[0])
                temp_dist[8].append(i[-5])
                temp_titles[8].append(i[1])
                temp_vectors[8].append(model.docvecs[i[0]])
    names = ['average growing unemployment districts','slow growing unemployment districts','fast growing unemployment districts',\
             'average growing agricultural districts','slow growing agricultural districts','fast growing agricultural districts',\
            'average growing non-agricultural districts','slow growing non-agricultural districts','fast growing non-agricultural districts']
    for i in range(9):
        titlee = 'Tsne '+names[i]+' in '+ collection_name + ' collection'
        #d,vectors = get_vectors(temp_vectors[i],temp_dist[i])
        distt = []
        vector = []
        for v in list(set(temp_dist[i])):
            distt.append(v)
            vector.append(model.docvecs[v])
        d,vectors = get_vectors_dist(vector,distt)       
        val = get_values(d)
        X,labels=kmeans(vectors)
        print(titlee,Counter(labels),val)
        path = 'clusters/'
        plot_cluster(X,labels,val,titlee,path,True)
        plot_cluster(X,labels,val,'Un '+titlee,path)
        
    